In [3]:
!pip install langchain_ollama langchain_core streamlit

/bin/bash: warning: setlocale: LC_ALL: cannot change locale (en_US.UTF-8)
  Using cached streamlit-1.45.1-py3-none-any.whl.metadata (8.9 kB)
  Using cached altair-5.5.0-py3-none-any.whl.metadata (11 kB)
  Using cached blinker-1.9.0-py3-none-any.whl.metadata (1.6 kB)
  Using cached cachetools-5.5.2-py3-none-any.whl.metadata (5.4 kB)
  Using cached click-8.2.0-py3-none-any.whl.metadata (2.5 kB)
  Using cached pillow-11.2.1-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (8.9 kB)
  Using cached protobuf-6.31.0-cp39-abi3-manylinux2014_x86_64.whl.metadata (593 bytes)
  Using cached pyarrow-20.0.0-cp311-cp311-manylinux_2_28_x86_64.whl.metadata (3.3 kB)
  Using cached toml-0.10.2-py2.py3-none-any.whl.metadata (7.1 kB)
  Using cached watchdog-6.0.0-py3-none-manylinux2014_x86_64.whl.metadata (44 kB)
  Using cached GitPython-3.1.44-py3-none-any.whl.metadata (13 kB)
  Using cached pydeck-0.9.1-py2.py3-none-any.whl.metadata (4.1 kB)
  Using cached narwhals-1.39.0-py3-none-any.whl.metadata (11 kB)
 

In [6]:
#!/usr/bin/env python3
import argparse
from langchain_ollama.chat_models import ChatOllama
from langchain_core.tools import tool
from langchain_core.messages import SystemMessage, HumanMessage
import requests

#
#   TOOL DEFINITION
#

@tool
def get_weather(latitude, longitude):
    """Fetches weather for given city"""

    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


#
#   RESPONSE FUNCTION
#

def get_response(prompt: str, use_function_calling: bool):
    messages = [
        SystemMessage("You are a helpful AI assistant. Answer concisely."),
        HumanMessage(prompt)
    ]

    if use_function_calling:
        model = ChatOllama(model="llama3.2").bind_tools([get_weather])
        # initial call
        res = model.invoke(messages)
        messages.append(res)

        # run any tool calls
        for tool_call in res.tool_calls:
            fn = tool_call["name"].lower()
            tool_fn = {"get_weather": get_weather}[fn]
            tool_msg = tool_fn.invoke(tool_call)
            messages.append(tool_msg)

        # final call with tool output
        res = model.invoke(messages)
        messages.append(res)
    else:
        model = ChatOllama(model="llama3.2")
        res = model.invoke(messages)
        messages.append(res)

    return res.content, messages

#
#   MAIN ENTRYPOINT
#

def main():
    parser = argparse.ArgumentParser(description="ChatOllama terminal client")
    parser.add_argument(
        "--prompt", "-p",
        type=str,
        required=True,
        help="The prompt to send to the assistant"
    )
    parser.add_argument(
        "--function-calling", "-f",
        action="store_true",
        help="Enable LangChain function calling (get_weather)"
    )
    args = parser.parse_args()

    response_text, chat_history = get_response(args.prompt, args.function_calling)

    print("\n=== Assistant response ===")
    print(response_text)

    print("\n=== Full chat messages ===")
    for msg in chat_history:
        role = getattr(msg, "role", msg.type)
        content = getattr(msg, "content", str(msg))
        print(f"[{role}] {content}")

if __name__ == "__main__":
    main()


usage: ipykernel_launcher.py [-h] --prompt PROMPT [--function-calling]
ipykernel_launcher.py: error: the following arguments are required: --prompt/-p


SystemExit: 2

/home/maiconkevyn/PycharmProjects/chatbot-pysus/.venv/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3678: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
